## Stahování dat z přehlídkových databází
### [OGLE II - archiv fotometrie](http://ogledb.astrouw.edu.pl/~ogle/photdb/phot_query.html)

is a Polish astronomical project based at the University of Warsaw that runs a long-term variability sky survey (1992-present). Main goals are the detection and classification of variable stars (pulsating and eclipsing), discoveries of the microlensing events, dwarf novae, studies of the Galaxy structure and the Magellanic Clouds. Since the project began in 1992, it has discovered multitude of extrasolar planets, together with a first planet discovered using transit method (OGLE-TR-56b) and gravitational microlensing method. The project from its inception is led by Prof. Andrzej Udalski. 

The main targets of the experiment are the Magellanic Clouds and the Galactic Bulge, because of the large number of intervening stars that can be used for microlensing during a stellar transit. Most of the observations have been made at the Las Campanas Observatory in Chile. Cooperating institutions include Princeton University and the Carnegie Institution. 

<img src="./images/oglephotometry.png" alt="Isaac Newton" style="width: 500px;"/>

Webové rozhraní SQL databáze [OGLE II](http://ogledb.astrouw.edu.pl/~ogle/photdb/phot_query.html) využívá technologie SQL dotazů, vrací nalezené křivky splňující zadané kriteria a zpřístupňuje naměřená data k jedotlivým objektům. 

## Web scrapping tutorial

In [1]:
import ipywidgets as widgets
from sqlalchemy import *
from IPython.display import display

Naším prvním úkolem bude stažení dat z članku [*BVI photometry of LMC Be stars*](https://academic.oup.com/mnras/article/361/3/1055/972019). Zásadním problémem je, že původní data k tomuto článku byla založena na starší verzi fotometrických dat (projekt OGLE) publikovaných například v katalogu proměnných hvězd projektu [Zebrun catalogue](http://articles.adsabs.harvard.edu/pdf/2001AcA....51..317Z). V tomto katalogu však byl zaveden jiný způsob katalogizace hvězd *OGLEXXXXXX-XXXXXXX* namísto současné označení LMC_SCXX_i_######. 

Načteme z naší databáze katalog Be hvězd *BeStarsCatalog.db* s již příslušným současným označením LMC_SCXX_i_######.

In [2]:
from sqlalchemy import *

engine = create_engine('sqlite:///BeStarsCatalog.db')
metadata = MetaData()
metadata.reflect(bind=engine)

conn = engine.connect()
sabogal = metadata.tables['Sabogal']
zebrun = metadata.tables['Zebrun']

selection = select([zebrun])
result = conn.execute(selection).fetchall()

Vytvoříme se aktuální list světelených křivek obsahujících číselné ID pole a hvězdy z výsledku obdrženého dotazem do databáze.

In [3]:
lightcurves_list = []
for row in result:
    fieldnumber = row['Field']
    starid = row['OGLEII']
    field = 'LMC_sc'+str(fieldnumber)
    parameters = {"field": field,"starid": starid}
    lightcurves_list.append(parameters)
print('Last Field: %s  StarID: %s' % (field,starid))

Last Field: LMC_sc4  StarID: 219917


Nyní je již vše připraveno pro aplikaci web scrappingu s použitím knihovny [Selenium](https://selenium-python.readthedocs.io/), s jejíž pomocí lze snadno automatizovat stahování dat z přehlídkového katalogu. Nejprve definujeme adresu dynamického webu, ze které chceme získat požadované informace. Pro správnou funkci knihovny je také nutné mít nainstalovaný driver prohlížeče, v našem případě [Chrome](https://chromedriver.chromium.org/) a definovat klíčové prvky formuláře, které budeme vyplňovat. Jejich jména zjistíme použitím vývojářských funkcí prohlížeče na zadané adrese.

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import sys

baseurl = "http://ogledb.astrouw.edu.pl/~ogle/photdb/phot_query.html"
browser = webdriver.Chrome('./chromedriver')  

browser.get(baseurl) #navigate to the page

db_target = browser.find_element_by_id("db_target3")

val_field = browser.find_element_by_name("val_field")
val_starid = browser.find_element_by_name("val_starid")

use_starid = browser.find_element_by_name("use_starid")
use_field = browser.find_element_by_name("use_field")

 

Máme-li definovány prvky HTML formuláře, přejdeme k jejich vyplněni hodnotami podle našich požadavků. Využijeme k tomu metod *send_keys*, *is_selected*, které náleží k příslušným elementům definovaným výše. A formulář odešleme metodou *click* příslušnou k elementu *Button*.

In [5]:
val_field.send_keys("LMC_sc10")
val_starid.send_keys("1006*")

if not(db_target.is_selected()):
    db_target.click()

if not(use_starid.is_selected()):
    use_starid.click()

if not(use_field.is_selected()):
    use_field.click()
    
submitButton = browser.find_element_by_xpath("//input[@type='submit' and @value='Submit Query']")
submitButton.click()

innerHTML = browser.execute_script("return document.body.innerHTML")

Nyní je třeba zpracovat HTML výstup generovaný jako odpověď na základě vyplněného HTML formuláře s pomocí knihovny [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). 

<img src="./images/resulttableview.png" alt="Isaac Newton" style="width: 800px;"/>

Část výsledku dotazu v HTML vypadá následovně
<img src="./images/resulttablehtml.png" alt="Isaac Newton" style="width: 600px;"/>

Dále extrahujeme z výsledků hledání hlavičku tabulky, která obsahuje názvy jednotlivých sloupců  a pro naši kontrolu si hlavičku vypíšeme

In [6]:
soup = BeautifulSoup(innerHTML, 'html.parser')
header = [th.find(text=True) for th in soup.findAll("th")]

# Find HTML element table
table = soup.find("table", {"border": "1"})
# Find all rows in the table
rows = table.findAll('tr')
# Find all hyperlinks in the table
links = table.findAll('a')

print(header)

['No', 'Field', 'StarID', 'StarCat', 'RA', 'Decl', 'Pgood', 'I', 'Isig', 'Imed', 'Imederr', 'Ndetect']


Hlavička odpovědi obsahuje jména jednotlivých sloupců výsledné tabulky a to podle jejich pořadí. Dále z výsledné tabulky extrahujeme odkazy na jednotlivé světelné křivky a již upravené řádky tabulky s jednotlivými parametry hvězdy

In [7]:
# Dictionary contains names for mapping of column names from table according to their positions
COL_MAP = {"Field": "field",
               "StarID": "starid",
               "RA": "ra",
               "Decl": "dec",
               "V": "v_mag",
               "I": "i_mag",
               "B": "b_mag"}

tmp_lcs = []

for link in links:
    tmp_lcs.append(link.get('href'))
    
res_rows = []
for tr in rows[1:]:
    cols = tr.findAll('td')
    res_cols = []
    for td in cols:
        res_cols.append(td.find(text=True))
    res_rows.append(res_cols)

def _parseHeader(header):
    cols_map = {}
    for i, col in enumerate(header):
        if col in list(COL_MAP.keys()):
            cols_map[COL_MAP[col]] = i
    return cols_map



Následně extrahujeme z tabulky samotné měření (fotometrii) vybrané hvězdy, měření jsou dostupná přes html link ve sloupci *StarId*. Jedná se o dynamický link, vytvořený při dotazu, jehož platnost je časově omezena. Vytvoříme si funkci, která nám zařidí stažení všech měření světelných křivek, vstupním parametrem je seznam obsahující všechny odkazy na měření, které program našel ve výsledku z dotazu do databáze.

Pro jednoduchou ilustraci funkčnosti našeho postupu zvolíme pouze jednu světelnou křivku, konkrétně 11-člen seznamu *tmp_lcs*. Po stažení této křivky si zobrazíme její část, abychom viděli strukturu dat.

In [8]:
def _getLc(lc_tmp):

        ROOT = "http://ogledb.astrouw.edu.pl/~ogle/photdb"
        url = "%s/" %ROOT +lc_tmp
        browser.get(url) #navigate to the page
        HTML = browser.execute_script("return document.body.innerHTML")
        try:
            soup = BeautifulSoup(HTML,'html.parser')
            data = soup.find("pre")
            starcurve = data.contents[0]

            return starcurve

        # TODO
        except:
            raise

data=_getLc(tmp_lcs[11])

import numpy as np

table = []
for line in data.splitlines():
    row = np.asarray(line.split())
    table.append(row)
    
#ogletable = pd.DataFrame(table,columns=["HJD","mag","error","bad points","Flag"])
# Namisto PANDAS knihovny zkusit ASTROPY


Tabulku s pozorováním můžeme také uložit do souboru pro další použití

In [10]:
file = open(r"lmc_test.dat","w+")
file.write(data)
file.close()

browser.quit()

## Automated download selected set of lightcurves

Nyní máme již připraveno vše pro automatizaci celého procesu. Zadání je následující. Máme předem známý seznam hvězd z přehlídky OGLE (většinou se jedná o hvězdy určitého typu, který nás zajímá - v našem případě *Be hvězdy*). Pro tento sezname chceme z webového rozhraní stáhnout všechny dostupné data. Problém lze rozdělit na tři procesy: *Dotaz*, *Extrakce dat* a *Uložení dat*. Napíšeme si tedy adekvátní procedury a začneme dotazem do databáze - procedura *query_data*, jejíž struktura byla již popsána výše. Samozřejmě, jako úplně první krok je třeba importovat příslušné knihovny a inicializovat driver browseru *Chrome*.

In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup
import sys

baseurl = "http://ogledb.astrouw.edu.pl/~ogle/photdb/phot_query.html"
browser = webdriver.Chrome('./chromedriver')

def query_data(field,starid):
    browser.get(baseurl) #navigate to the page
    db_target = browser.find_element_by_id("db_target3")
    val_field = browser.find_element_by_name("val_field")
    val_starid = browser.find_element_by_name("val_starid")
    use_starid = browser.find_element_by_name("use_starid")
    use_field = browser.find_element_by_name("use_field")
    
    val_field.send_keys(field)
    val_starid.send_keys(starid)

    if not(db_target.is_selected()):
        db_target.click()
    if not(use_starid.is_selected()):
        use_starid.click()
    if not(use_field.is_selected()):
        use_field.click()
    
    submitButton = browser.find_element_by_xpath("//input[@type='submit' and @value='Submit Query']")
    submitButton.click()
    innerHTML = browser.execute_script("return document.body.innerHTML")
    return innerHTML


Následuje procedura *process_data*, která zpracovává výsledek dotazu, extrahuje informace o hvězdě včetně dynamického odkazu na dočasný soubor s naměřenými daty. Soubor stáhneme a uložíme do příslušné proměnné. Struktura opět odpovídá již výše popsanému postupu

In [12]:
def process_data(html):
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find("table", {"border": "1"})
    links = table.find("a", href=True)
    tmp_link = links['href']
    root = 'http://ogledb.astrouw.edu.pl/~ogle/photdb/'
    url = root + tmp_link
    browser.get(url)
    textHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(textHTML,'html.parser')
    data = soup.find("pre")

    if not data:
        return []
    else:
        return data.contents[0]


Nakonec data uložíme do klasického textového souboru

In [13]:
def save_data(filename,content):
    file = open(filename,'w+')
    file.write(content)
    file.close()
    

Všechny procedury máme naprogramované, zbývá už jen cyklus který provede dotaz, extrakci a uložení pro všechny členy vybraného seznamu hvězd. A aby to bylo ještě zábavnější a především hezčí, umístíme do notebooku i widget *ProgressBar*, který nám pěkně graficky ukáže, jak jsme se stahováním daleko.

In [14]:
f = widgets.FloatProgress(min=0, max=2446,description='Download progress')
counter_text = widgets.Text(
    value='',
    placeholder='Info',
    description='Counter:',
    disabled=True)
info_text = widgets.Text(
    value='',
    placeholder='Info',
    description='File: ',
    disabled=True)

display(f)

errors = []
for counter,lc in enumerate(lightcurves_list):
    field = lc["field"]
    starid = lc["starid"]
    html = query_data(field,starid)
    content = process_data(html)
    filename = lc["field"].lower()+'_i_'+lc["starid"]+'.dat'
    if content:
        save_data(filename,content)
    else:
        print("No good DIA photometry for this object")
        errors.append(filename)
    if counter > 10:
        browser.quit()
        sys.exit()
    
    info_text.value = filename
    counter_text.value = str(counter)
    f.value = counter

browser.quit()

FloatProgress(value=0.0, description='Download progress', max=2446.0)

KeyboardInterrupt: 